In [1]:
import os
import numpy as np
import json
import argparse
import h5py
from pathlib import Path

import config
from GPT import GPT
from Decoder import Decoder, Hypothesis
from LanguageModel import LanguageModel
from EncodingModel import EncodingModel
from StimulusModel import StimulusModel, get_lanczos_mat, affected_trs, LMFeatures
from utils_stim import predict_word_rate, predict_word_times

/mnt/sphere/projects/simon/semantic-decoding/pyenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/sphere/projects/simon/semantic-decoding/pyenv/lib/python3.10/site-packages/torch/__init__.py:747: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:431.)
  _C._set_default_tensor_type(t)


In [2]:
gpt_checkpoint = "perceived"
word_rate_voxels = "auditory"
subject = 'S1'
experiment = 'perceived_speech'
task = 'wheretheressmoke'

In [3]:
# load responses
hf = h5py.File(os.path.join(config.DATA_TEST_DIR, "test_response", subject, experiment, task + ".hf5"), "r")
resp = np.nan_to_num(hf["data"][:])
hf.close()
resp.shape

(291, 81126)

In [4]:
# load gpt
with open(os.path.join(config.DATA_LM_DIR, gpt_checkpoint, "vocab.json"), "r") as f:
    gpt_vocab = json.load(f)
with open(os.path.join(config.DATA_LM_DIR, "decoder_vocab.json"), "r") as f:
    decoder_vocab = json.load(f)
gpt = GPT(path = os.path.join(config.DATA_LM_DIR, gpt_checkpoint, "model"), vocab = gpt_vocab, device = "cuda:5")
features = LMFeatures(model = gpt, layer = config.GPT_LAYER, context_words = config.GPT_WORDS)
lm = LanguageModel(gpt, decoder_vocab, nuc_mass = config.LM_MASS, nuc_ratio = config.LM_RATIO)
len(gpt_vocab), len(decoder_vocab)

(17379, 6867)

In [5]:
# load models
load_location = os.path.join(config.MODEL_DIR, subject)
word_rate_model = np.load(os.path.join(load_location, "word_rate_model_%s_orig.npz" % word_rate_voxels), allow_pickle = True)
encoding_model = np.load(os.path.join(load_location, "encoding_model_%s_orig.npz" % gpt_checkpoint))
weights = encoding_model["weights"]
noise_model = encoding_model["noise_model"]
tr_stats = encoding_model["tr_stats"]
word_stats = encoding_model["word_stats"]
em = EncodingModel(resp, weights, encoding_model["voxels"], noise_model, device = "cuda:6")
em.set_shrinkage(config.NM_ALPHA)
assert task not in encoding_model["stories"]

In [6]:
list(encoding_model.keys()), list(word_rate_model.keys())

(['weights',
  'noise_model',
  'alphas',
  'voxels',
  'stories',
  'tr_stats',
  'word_stats'],
 ['weights', 'mean_rate', 'voxels'])

In [7]:
word_rate_model['weights'].shape, word_rate_model['mean_rate'], word_rate_model['voxels'].shape

((5724, 1), array(5.45454908), (1431,))

In [8]:
weights.shape, noise_model.shape, tr_stats.shape, word_stats.shape, encoding_model["voxels"].shape

((3072, 81126), (10000, 10000), (2, 768), (2, 768), (10000,))

In [9]:
# predict word times
word_rate = predict_word_rate(resp, word_rate_model["weights"], word_rate_model["voxels"], word_rate_model["mean_rate"])
word_rate.shape

(291,)

In [10]:
word_times, tr_times = predict_word_times(word_rate, resp, starttime = -10)
word_times.shape, tr_times.shape

((1589,), (291,))

In [11]:
lanczos_mat = get_lanczos_mat(word_times, tr_times)
lanczos_mat.shape

/mnt/sphere/projects/simon/semantic-decoding/decoding/utils_ridge/interpdata.py:159: RuntimeWarning: invalid value encountered in divide
  val = window * np.sin(np.pi*t) * np.sin(np.pi*t/window) / (np.pi**2 * t**2)


(291, 1589)

In [12]:
# decode responses
decoder = Decoder(word_times, config.WIDTH)
sm = StimulusModel(lanczos_mat, tr_stats, word_stats[0], device = "cuda:4")
for sample_index in range(120):
    trs = affected_trs(decoder.first_difference(), sample_index, lanczos_mat)
    ncontext = decoder.time_window(sample_index, config.LM_TIME, floor = 5)
    beam_nucs = lm.beam_propose(decoder.beam, ncontext)
    for c, (hyp, nextensions) in enumerate(decoder.get_hypotheses()):
        # print(c, len(hyp.words), nextensions, len(beam_nucs))
        nuc, logprobs = beam_nucs[c]
        if len(nuc) < 1: continue
        extend_words = [hyp.words + [x] for x in nuc]
        extend_embs = list(features.extend(extend_words, verbose=False))
        stim = sm.make_variants(sample_index, hyp.embs, extend_embs, trs)
        likelihoods = em.prs(stim, trs)
        local_extensions = [Hypothesis(parent = hyp, extension = x) for x in zip(nuc, logprobs, extend_embs)]
        decoder.add_extensions(local_extensions, likelihoods, nextensions)
    decoder.extend(verbose = True)

['i']
['i', 'have']
['she', 'said', 'she']
['she', 'would', 'always', 'say']
['she', 'would', 'always', 'say', 'she']
['she', 'would', 'say', 'she', "wasn't", 'feeling']
['she', 'would', 'always', 'say', 'she', "wasn't", 'feeling']
['she', 'said', 'she', 'was', 'a', 'little', 'stressed', 'out']
['she', 'said', 'she', 'was', 'a', 'little', 'stressed', 'out', 'she']
['she', 'said', 'she', 'was', 'a', 'bit', 'drunk', 'and', 'not', 'ready']
['she', 'said', 'she', 'was', 'a', 'bit', 'drunk', 'and', 'it', "wasn't", 'safe']
['she', 'said', 'she', 'was', 'a', 'bit', 'drunk', 'and', 'it', 'was', 'too', 'hot']
['she', 'would', 'say', 'she', 'had', 'been', 'crying', 'because', 'she', "wasn't", 'hungry', 'or', 'anything']
['she', 'would', 'say', 'she', 'had', 'been', 'crying', 'because', 'she', "wasn't", 'hungry', 'or', 'anything', 'like']
['she', 'said', 'she', 'was', 'just', 'tired', 'from', 'working', 'and', 'i', "couldn't", 'ask', 'for', 'more', 'than']
['she', 'said', 'she', 'was', 'just', 't

In [115]:
[local_extensions[i].words for i in range(6)]

[['i'], ['we'], ['she'], ['he'], ['they'], ['it']]

In [111]:
likelihoods

array([-21884.957, -21884.965, -21884.98 , -21884.982, -21884.98 ,
       -21884.986], dtype=float32)

In [103]:
stim.cpu().numpy().shape

(6, 6, 3072)

In [99]:
len(extend_embs), extend_embs[0].shape

(6, (768,))

In [93]:
features

In [64]:
decoder.first_difference()

0